In [1]:
!pip install spacy-langdetect
!pip install language-detector
!pip install symspellpy
!pip install sentence-transformers
!pip install stop-words
!pip install pyLDAvis

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
#covid-papers-browser/models/scibert-nli/

# Topic Modeling with BERT, LDA, and Clustering
#### Latent Dirichlet Allocation (LDA) probabilistic topic assignment and pre-trained sentence embeddings from BERT/RoBERTa

**Insired by:** 

Shoa, S. (2020). **Contextual Topic Identification: Identifying meaningful topics for sparse Steam reviews**. Medium. Available at: https://blog.insightdatascience.com/contextual-topic-identification-4291d256a032 [Accessed 25 Mar. 2020].

## Model Deep Dive

The author used: 

* LDA for probabilistic topic assignment vector.
* Bert for sentence embedding vector.

1. Concatenated both LDA and Bert vectors with a weight hyperparameter to balance the relative importance of information from each source.
2. Used autoencoder to learn a lower dimensional latent space representation of the concatenated vector.

* The assumption is that the concatendate vector shoul have a manifold shaep in the high dimensional space. 
* USed clustering on the latent space representations to get topics. 



![Contextual Topic Identification model design](https://miro.medium.com/max/1410/1*OKCYnB-JbGq1NDwNSKd5Zw.png)

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/drive/My Drive/tweets_analysis/cleaned_tweets_files'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.


/content/drive/My Drive/tweets_analysis/cleaned_tweets_files/en_geo_2020-02-01.csv
/content/drive/My Drive/tweets_analysis/cleaned_tweets_files/en_geo_2020-02-02.csv
/content/drive/My Drive/tweets_analysis/cleaned_tweets_files/en_geo_2020-02-03.csv
/content/drive/My Drive/tweets_analysis/cleaned_tweets_files/en_geo_2020-02-04.csv
/content/drive/My Drive/tweets_analysis/cleaned_tweets_files/en_geo_2020-02-05.csv
/content/drive/My Drive/tweets_analysis/cleaned_tweets_files/en_geo_2020-02-06.csv
/content/drive/My Drive/tweets_analysis/cleaned_tweets_files/en_geo_2020-02-07.csv
/content/drive/My Drive/tweets_analysis/cleaned_tweets_files/en_geo_2020-02-08.csv
/content/drive/My Drive/tweets_analysis/cleaned_tweets_files/en_geo_2020-02-09.csv
/content/drive/My Drive/tweets_analysis/cleaned_tweets_files/en_geo_2020-02-10.csv
/content/drive/My Drive/tweets_analysis/cleaned_tweets_files/en_geo_2020-02-11.csv
/content/drive/My Drive/tweets_analysis/cleaned_tweets_files/en_geo_2020-02-12.csv
/con

In [6]:
#importing all dependencies.
import os
import json
import pandas as pd
from tqdm import tqdm
import numpy as np
from nltk.corpus import wordnet
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords 
#from geotext import GeoText
#/kaggle/input

## Upload Data

**Data pipeline (from development to deployment**)

![Data pipeline (from development to deployment)](https://miro.medium.com/max/1348/1*Cdp4y1tfMxqoj96o6lUdFg.png)


Source:Shoa 

In [7]:
#https://www.kaggle.com/jieyang0311/covid-19-topic-modeling-lda
#load library
import os
import pandas as pd
import numpy as np
import gensim
from gensim.utils import simple_preprocess
from gensim import corpora, models
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

import datetime
import time
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import nltk
import glob

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [8]:
tweets_df = pd.concat(map(pd.read_csv, glob.glob('/content/drive/My Drive/tweets_analysis/cleaned_tweets_files/*.csv')))
print(tweets_df.shape)

(791511, 3)


In [9]:
#Sentiment Analysis
from textblob import TextBlob
tweets_df['sentiment'] = tweets_df['full_text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)  #-1 to 1

In [12]:
india_based_user_df = tweets_df.loc[tweets_df['is_user_india_based'].apply(lambda x:bool(x))]
outside_india_based_user_df = tweets_df.loc[~tweets_df['is_user_india_based'].apply(lambda x:bool(x))]

In [13]:
display(tweets_df.shape)
display(india_based_user_df.shape)
display(outside_india_based_user_df.shape)

(791511, 4)

(104310, 4)

(687201, 4)

In [14]:
tweets_df.head()

,tweet_id,full_text,is_user_india_based,sentiment
0,1223489863552421888,normala sister talkin of viability gap funding...,0,0.25
1,1223729344432001024,air indias 747 carrying evacuees from wuhan ha...,0,0.00
2,1223490838589632512,indian already airlifted this is india isme sa...,0,0.00
3,1223491079628115968,one takeaway from the coronavirus dont punish ...,0,0.00
4,1223491311371575296,an air india flight carrying 324 indian nation...,0,0.00


In [15]:
##extract the abstract to pandas 
india_based_documents = india_based_user_df.iloc[:, [1,3]]
india_based_documents=india_based_documents.reset_index()
india_based_documents.drop("index", inplace = True, axis = 1)

##create pandas data frame with all abstracts, use as input corpus
india_based_documents["index"] = india_based_documents.index.values
india_based_documents.head(3)

,full_text,sentiment,index
0,relieved to see ai 1349 return safely to with ...,0.500000,0
1,everytime you abuse for its deficient services...,-0.200000,1
2,air india special flight to evacuate indian ci...,0.357143,2


In [16]:
##extract the abstract to pandas 
outside_india_based_documents = outside_india_based_user_df.iloc[:, [1,3]]
outside_india_based_documents=outside_india_based_documents.reset_index()
outside_india_based_documents.drop("index", inplace = True, axis = 1)

##create pandas data frame with all abstracts, use as input corpus
outside_india_based_documents["index"] = outside_india_based_documents.index.values
outside_india_based_documents.head(3)

,full_text,sentiment,index
0,normala sister talkin of viability gap funding...,0.25,0
1,air indias 747 carrying evacuees from wuhan ha...,0.00,1
2,indian already airlifted this is india isme sa...,0.00,2


## Utils

In [17]:
from collections import Counter
from sklearn.metrics import silhouette_score
import umap
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from gensim.models.coherencemodel import CoherenceModel
import numpy as np
import os


def get_topic_words(token_lists, labels, k=None):
    """
    get top words within each topic from clustering results
    """
    if k is None:
        k = len(np.unique(labels))
    topics = ['' for _ in range(k)]
    for i, c in enumerate(token_lists):
        topics[labels[i]] += (' ' + ' '.join(c))
    word_counts = list(map(lambda x: Counter(x.split()).items(), topics))
    # get sorted word counts
    word_counts = list(map(lambda x: sorted(x, key=lambda x: x[1], reverse=True), word_counts))
    # get topics
    topics = list(map(lambda x: list(map(lambda x: x[0], x[:10])), word_counts))

    return topics

def get_coherence(model, token_lists, measure='c_v'):
    """
    Get model coherence from gensim.models.coherencemodel
    :param model: Topic_Model object
    :param token_lists: token lists of docs
    :param topics: topics as top words
    :param measure: coherence metrics
    :return: coherence score
    """
    if model.method == 'LDA':
        cm = CoherenceModel(model=model.ldamodel, texts=token_lists, corpus=model.corpus, dictionary=model.dictionary,
                            coherence=measure)
    else:
        topics = get_topic_words(token_lists, model.cluster_model.labels_)
        cm = CoherenceModel(topics=topics, texts=token_lists, corpus=model.corpus, dictionary=model.dictionary,
                            coherence=measure)
    return cm.get_coherence()

def get_silhouette(model):
    """
    Get silhouette score from model
    :param model: Topic_Model object
    :return: silhouette score
    """
    if model.method == 'LDA':
        return
    lbs = model.cluster_model.labels_
    vec = model.vec[model.method]
    return silhouette_score(vec, lbs)

def plot_proj(embedding, lbs):
    """
    Plot UMAP embeddings
    :param embedding: UMAP (or other) embeddings
    :param lbs: labels
    """
    n = len(embedding)
    counter = Counter(lbs)
    for i in range(len(np.unique(lbs))):
        plt.plot(embedding[:, 0][lbs == i], embedding[:, 1][lbs == i], '.', alpha=0.5,
                 label='cluster {}: {:.2f}%'.format(i, counter[i] / n * 100))
    plt.legend(loc = 'best')
    plt.grid(color ='grey', linestyle='-',linewidth = 0.25)


def visualize(model):
    """
    Visualize the result for the topic model by 2D embedding (UMAP)
    :param model: Topic_Model object
    """
    if model.method == 'LDA':
        return

    if model.prefix == None:
        model.prefix = model.method

    reducer = umap.UMAP()
    print('Calculating UMAP projection ...')
    vec_umap = reducer.fit_transform(model.vec[model.method])
    print('Calculating UMAP projection. Done!')
    plot_proj(vec_umap, model.cluster_model.labels_)
    dr = '/content/drive/My Drive/tweets_analysis/working/contextual_topic_identification/docs/images/{}/{}/{}'.format(model.prefix, model.method, model.id)
    if not os.path.exists(dr):
        os.makedirs(dr)
    plt.savefig('/content/drive/My Drive/tweets_analysis/working/{}/2D_vis'.format(model.prefix))

def get_wordcloud(model, token_lists, topic):
    """
    Get word cloud of each topic from fitted model
    :param model: Topic_Model object
    :param sentences: preprocessed sentences from docs
    """
    if model.method == 'LDA':
        return

    if model.prefix == None:
        model.prefix = model.method

    print('Getting wordcloud for topic {} ...'.format(topic))
    lbs = model.cluster_model.labels_
    tokens = ' '.join([' '.join(_) for _ in np.array(token_lists)[lbs == topic]])

    wordcloud = WordCloud(width=800, height=560,
                          background_color='white', collocations=False,
                          min_font_size=10).generate(tokens)

    # plot the WordCloud image
    plt.figure(figsize=(8, 5.6), facecolor=None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad=0)
    dr = '/content/drive/My Drive/tweets_analysis/working/{}/{}/{}'.format(model.prefix, model.method, model.id)
    if not os.path.exists(dr):
        os.makedirs(dr)
    plt.savefig('/content/drive/My Drive/tweets_analysis/working' + '/' + model.prefix + '/Topic' + str(topic) + '_wordcloud')
    print('Getting wordcloud for topic {}. Done!'.format(topic))

## Preprocessing 

In [18]:
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
import re
import nltk
from nltk.tokenize import word_tokenize
from language_detector import detect_language

import pkg_resources
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
if sym_spell.word_count:
    pass
else:
    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)


###################################
#### sentence level preprocess ####
###################################

# lowercase + base filter
# some basic normalization
def f_base(s):
    """
    :param s: string to be processed
    :return: processed string: see comments in the source code for more info
    """
    # normalization 1: xxxThis is a --> xxx. This is a (missing delimiter)
    s = re.sub(r'([a-z])([A-Z])', r'\1\. \2', s)  # before lower case
    # normalization 2: lower case
    s = s.lower()
    # normalization 3: "&gt", "&lt"
    s = re.sub(r'&gt|&lt', ' ', s)
    # normalization 4: letter repetition (if more than 2)
    s = re.sub(r'([a-z])\1{2,}', r'\1', s)
    # normalization 5: non-word repetition (if more than 1)
    s = re.sub(r'([\W+])\1{1,}', r'\1', s)
    # normalization 6: string * as delimiter
    s = re.sub(r'\*|\W\*|\*\W', '. ', s)
    # normalization 7: stuff in parenthesis, assumed to be less informal
    s = re.sub(r'\(.*?\)', '. ', s)
    # normalization 8: xxx[?!]. -- > xxx.
    s = re.sub(r'\W+?\.', '.', s)
    # normalization 9: [.?!] --> [.?!] xxx
    s = re.sub(r'(\.|\?|!)(\w)', r'\1 \2', s)
    # normalization 10: ' ing ', noise text
    s = re.sub(r' ing ', ' ', s)
    # normalization 11: noise text
    s = re.sub(r'product received for free[.| ]', ' ', s)
    # normalization 12: phrase repetition
    s = re.sub(r'(.{2,}?)\1{1,}', r'\1', s)

    return s.strip()


# language detection
def f_lan(s):
    """
    :param s: string to be processed
    :return: boolean (s is English)
    """

    # some reviews are actually english but biased toward french
    return detect_language(s) in {'English'}
    # return detect_language(s) in {'English', 'French','Spanish','Chinese'}


###############################
#### word level preprocess ####
###############################

# filtering out punctuations and numbers
def f_punct(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with punct and number filter out
    """
    return [word for word in w_list if word.isalpha()]


# selecting nouns
def f_noun(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with only nouns selected
    """
    return [word for (word, pos) in nltk.pos_tag(w_list) if pos[:2] == 'NN']


# typo correction
def f_typo(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with typo fixed by symspell. words with no match up will be dropped
    """
    w_list_fixed = []
    for word in w_list:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=3)
        if suggestions:
            w_list_fixed.append(suggestions[0].term)
        else:
            pass
            # do word segmentation, deprecated for inefficiency
            # w_seg = sym_spell.word_segmentation(phrase=word)
            # w_list_fixed.extend(w_seg.corrected_string.split())
    return w_list_fixed


# stemming if doing word-wise
p_stemmer = PorterStemmer()


def f_stem(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with stemming
    """
    return [p_stemmer.stem(word) for word in w_list]


# filtering out stop words
# create English stop words list

stop_words = (list(
    set(get_stop_words('en'))
    # |set(get_stop_words('es'))
    # |set(get_stop_words('de'))
    # |set(get_stop_words('it'))
    # |set(get_stop_words('ca'))
    #|set(get_stop_words('cy'))
    # |set(get_stop_words('pt'))
    #|set(get_stop_words('tl'))
    # |set(get_stop_words('pl'))
    #|set(get_stop_words('et'))
    # |set(get_stop_words('da'))
    # |set(get_stop_words('ru'))
    #|set(get_stop_words('so'))
    # |set(get_stop_words('sv'))
    # |set(get_stop_words('sk'))
    #|set(get_stop_words('cs'))
    # |set(get_stop_words('nl'))
    #|set(get_stop_words('sl'))
    #|set(get_stop_words('no'))
    #|set(get_stop_words('zh-cn'))
))





def f_stopw(w_list):
    """
    filtering out stop words
    """
    return [word for word in w_list if word not in stop_words]


def preprocess_sent(rw):
    """
    Get sentence level preprocessed data from raw review texts
    :param rw: review to be processed
    :return: sentence level pre-processed review
    """
    s = f_base(rw)
    if not f_lan(s):
        return None
    return s


def preprocess_word(s):
    """
    Get word level preprocessed data from preprocessed sentences
    including: remove punctuation, select noun, fix typo, stem, stop_words
    :param s: sentence to be processed
    :return: word level pre-processed review
    """
    if not s:
        return None
    w_list = word_tokenize(s)
    w_list = f_punct(w_list)
    w_list = f_noun(w_list)
    w_list = f_typo(w_list)
    w_list = f_stem(w_list)
    w_list = f_stopw(w_list)

    return w_list

## Autoencoder

In [19]:
import keras
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')


class Autoencoder:
    """
    Autoencoder for learning latent space representation
    architecture simplified for only one hidden layer
    """

    def __init__(self, latent_dim=32, activation='relu', epochs=200, batch_size=128):
        self.latent_dim = latent_dim
        self.activation = activation
        self.epochs = epochs
        self.batch_size = batch_size
        self.autoencoder = None
        self.encoder = None
        self.decoder = None
        self.his = None

    def _compile(self, input_dim):
        """
        compile the computational graph
        """
        input_vec = Input(shape=(input_dim,))
        encoded = Dense(self.latent_dim, activation=self.activation)(input_vec)
        decoded = Dense(input_dim, activation=self.activation)(encoded)
        self.autoencoder = Model(input_vec, decoded)
        self.encoder = Model(input_vec, encoded)
        encoded_input = Input(shape=(self.latent_dim,))
        decoder_layer = self.autoencoder.layers[-1]
        self.decoder = Model(encoded_input, self.autoencoder.layers[-1](encoded_input))
        self.autoencoder.compile(optimizer='adam', loss=keras.losses.mean_squared_error)

    def fit(self, X):
        if not self.autoencoder:
            self._compile(X.shape[1])
        X_train, X_test = train_test_split(X)
        self.his = self.autoencoder.fit(X_train, X_train,
                                        epochs=self.epochs,
                                        batch_size=self.batch_size,
                                        shuffle=True,
                                        validation_data=(X_test, X_test), verbose=0)

## Model

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from gensim import corpora
import gensim
import numpy as np
#from Autoencoder import *
#from preprocess import *
from datetime import datetime
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import torch

def preprocess(docs, sentiment_scores, samp_size=None):
    """
    Preprocess the data
    """
    if not samp_size:
        samp_size = 100

    print('Preprocessing raw texts ...')
    n_docs = len(docs)
    sentences = []  # sentence level preprocessed
    sentiments = []
    token_lists = []  # word level preprocessed
    idx_in = []  # index of sample selected
    #     samp = list(range(100))
    samp = np.random.choice(n_docs, samp_size)
    print('Total samples : {}'.format(len(samp)))
    for i, idx in enumerate(samp):
        sentence = preprocess_sent(docs[idx])
        sentiment = sentiment_scores[idx]
        token_list = preprocess_word(sentence)
        if token_list:
            idx_in.append(idx)
            sentences.append(sentence)
            sentiments.append(sentiment)
            token_lists.append(token_list)
        print('{} %'.format(str(np.round((i + 1) / len(samp) * 100, 2))), end='\r')
    print('Preprocessing raw texts. Done!')
    return sentences, sentiments, token_lists, idx_in


# define model object
class Topic_Model:
    def __init__(self, k=10, method='TFIDF', prefix=None, latent_dim=32, activation='relu', epochs=200, batch_size=128):
        """
        :param k: number of topics
        :param method: method chosen for the topic model
        """
        if method not in {'TFIDF', 'LDA', 'BERT', 'LDA_BERT'}:
            raise Exception('Invalid method!')
        self.k = k
        self.dictionary = None
        self.corpus = None
        # self.stopwords = None
        self.cluster_model = None
        self.ldamodel = None
        self.vec = {}
        self.gamma = 15  # parameter for reletive importance of lda
        self.method = method
        self.AE = None
        self.id = method + '_' + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        self.prefix = prefix
        self.latent_dim=latent_dim
        self.activation=activation
        self.epochs=epochs
        self.batch_size=batch_size

    #Scale the 'sentiment' vector
    #Sentiment varies from -1 to +1
    def sentiment(self, x):
        if x < 0.04:
            return 0
        elif x > 0.04:
            return 1
        else:
            return 0.5

    #Mean Pooling - Take attention mask into account for correct averaging
    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask
    
    #Max Pooling - Take the max value over time for every dimension
    def max_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        token_embeddings[input_mask_expanded == 0] = -1e9  # Set padding tokens to large negative value
        max_over_time = torch.max(token_embeddings, 1)[0]
        return max_over_time

    def encode_bert_vec(self, sentences):
        print('Getting vector representations for BERT ...')
        #  # model = SentenceTransformer('bert-base-nli-max-tokens')
        #   model = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')
        #   vec = np.array(model.encode(sentences, show_progress_bar=True))
        tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/roberta-large-nli-stsb-mean-tokens")
        model = AutoModel.from_pretrained("sentence-transformers/roberta-large-nli-stsb-mean-tokens")

        #Tokenize sentences
        encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')

        #Compute token embeddings
        with torch.no_grad():
            model_output = model(**encoded_input)

        #Perform pooling. In this case, max pooling
        sentence_embeddings = self.mean_pooling(model_output, encoded_input['attention_mask'])

        print('Getting vector representations for BERT. Done!')

        vec = np.array(sentence_embeddings)

        return vec

    def vectorize(self, sentences, token_lists, sentiments, method=None):
        """
        Get vector representations from selected methods
        """
        # Default method
        if method is None:
            method = self.method

        # turn tokenized documents into a id <-> term dictionary
        self.dictionary = corpora.Dictionary(token_lists)
        # convert tokenized documents into a document-term matrix
        self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]

        if method == 'TFIDF':
            print('Getting vector representations for TF-IDF ...')
            tfidf = TfidfVectorizer()
            vec = tfidf.fit_transform(sentences)
            print('Getting vector representations for TF-IDF. Done!')
            return vec

        elif method == 'LDA':
            print('Getting vector representations for LDA ...')
            if not self.ldamodel:
                self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics=self.k, id2word=self.dictionary,
                                                                passes=20)

            def get_vec_lda(model, corpus, k):
                """
                Get the LDA vector representation (probabilistic topic assignments for all documents)
                :return: vec_lda with dimension: (n_doc * n_topic)
                """
                n_doc = len(corpus)
                vec_lda = np.zeros((n_doc, k))
                for i in range(n_doc):
                    # get the distribution for the i-th document in corpus
                    for topic, prob in model.get_document_topics(corpus[i]):
                        vec_lda[i, topic] = prob

                return vec_lda

            vec = get_vec_lda(self.ldamodel, self.corpus, self.k)
            print('Getting vector representations for LDA. Done!')
            return vec

        elif method == 'BERT':

            # print('Getting vector representations for BERT ...')
            # # model = SentenceTransformer('bert-base-nli-max-tokens')
            # model = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')
            # vec = np.array(model.encode(sentences, show_progress_bar=True))
            # print('Getting vector representations for BERT. Done!')
            vec = self.encode_bert_vec(sentences)
            return vec

             
        elif method == 'LDA_BERT':
        #else:
            vec_lda = self.vectorize(sentences, token_lists, sentiments, method='LDA')
            vec_bert = self.vectorize(sentences, token_lists, sentiments, method='BERT')

            vec_ldabert = np.c_[vec_lda * self.gamma, vec_bert]

            vec_ldabert_df = pd.DataFrame(vec_ldabert)

            vec_ldabert_df = pd.concat([vec_ldabert_df, pd.Series(sentiments).apply(lambda x: self.sentiment(x))], axis=1)

            vec_ldabert = vec_ldabert_df.to_numpy()

            self.vec['LDA_BERT_FULL'] = vec_ldabert
            if not self.AE:
                self.AE = Autoencoder(self.latent_dim, self.activation, self.epochs, self.batch_size)
                print('Fitting Autoencoder ...')
                self.AE.fit(vec_ldabert)
                print('Fitting Autoencoder Done!')
            vec = self.AE.encoder.predict(vec_ldabert)
            return vec

    def fit(self, sentences, token_lists, sentiments, method=None, m_clustering=None):
        """
        Fit the topic model for selected method given the preprocessed data
        :docs: list of documents, each doc is preprocessed as tokens
        :return:
        """
        # Default method
        if method is None:
            method = self.method
        # Default clustering method
        if m_clustering is None:
            m_clustering = KMeans

        # turn tokenized documents into a id <-> term dictionary
        if not self.dictionary:
            self.dictionary = corpora.Dictionary(token_lists)
            # convert tokenized documents into a document-term matrix
            self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]

        ####################################################
        #### Getting ldamodel or vector representations ####
        ####################################################

        if method == 'LDA':
            if not self.ldamodel:
                print('Fitting LDA ...')
                self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics=self.k, id2word=self.dictionary,
                                                                passes=20)
                print('Fitting LDA Done!')
        else:
            print('Clustering embeddings ...')
            self.cluster_model = m_clustering(self.k)
            self.vec[method] = self.vectorize(sentences, token_lists, sentiments, method)
            self.cluster_model.fit(self.vec[method])
            print('Clustering embeddings. Done!')

    def predict(self, sentences, token_lists, out_of_sample=None):
        """
        Predict topics for new_documents
        """
        # Default as False
        out_of_sample = out_of_sample is not None

        if out_of_sample:
            corpus = [self.dictionary.doc2bow(text) for text in token_lists]
            if self.method != 'LDA':
                vec = self.vectorize(sentences, token_lists)
                print(vec)
        else:
            corpus = self.corpus
            vec = self.vec.get(self.method, None)

        if self.method == "LDA":
            lbs = np.array(list(map(lambda x: sorted(self.ldamodel.get_document_topics(x),
                                                     key=lambda x: x[1], reverse=True)[0][0],
                                    corpus)))
        else:
            lbs = self.cluster_model.predict(vec)
        return lbs

## Training

In [21]:
#from model import *
#from utils import *

import pandas as pd
import pickle
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore', category=Warning)

import argparse

#def model(): #:if __name__ == '__main__':

def main(documents, samp_size, prefix=None, ntopic=10, method='TFIDF', 
         latent_dim=32, activation='relu', epochs=200, batch_size=128, 
         show_visualize=True):
    # ntopic = 10
    
    #parser = argparse.ArgumentParser(description='contextual_topic_identification tm_test:1.0')

    #parser.add_argument('--fpath', default='/kaggle/working/train.csv')
    #parser.add_argument('--ntopic', default=10,)
    #parser.add_argument('--method', default='TFIDF')
    #parser.add_argument('--samp_size', default=20500)
    
    #args = parser.parse_args()

    data = documents #pd.read_csv('/kaggle/working/train.csv')
    data = data.fillna('')  # only the comments has NaN's
    rws = data.full_text
    sentiment_scores = data.sentiment
    sentences, sentiments, token_lists, idx_in = preprocess(rws, sentiment_scores, samp_size=samp_size)
    print('Total preprocessed sentences : {}'.format(len(sentences)))
    print('Total associated sentiments : {}'.format(len(sentiments)))
    # Define the topic model object
    #tm = Topic_Model(k = 10), method = TFIDF)
    tm = Topic_Model(k = ntopic, method = method, prefix=prefix, 
                     latent_dim=latent_dim, activation=activation, 
                     epochs=epochs, batch_size=batch_size)
    # Fit the topic model by chosen method
    tm.fit(sentences, token_lists, sentiments)
    # Evaluate using metrics
    # with open("/content/drive/My Drive/tweets_analysis/working/{}.file".format(tm.id), "wb") as f:
    #     pickle.dump(tm, f, pickle.HIGHEST_PROTOCOL)
    # file_pi = open("/content/drive/My Drive/tweets_analysis/working/{}.file".format(tm.id), "w") 
    # pickle.dump(tm, file_pi)

    coherence_sc = get_coherence(tm, token_lists, 'c_v')
    silhouette_sc = get_silhouette(tm)

    print('Coherence:', coherence_sc)
    print('Silhouette Score:', silhouette_sc)
    # visualize and save img
    if show_visualize:
        visualize(tm)

        for i in range(tm.k):
            get_wordcloud(tm, token_lists, i)

    return (tm, coherence_sc, silhouette_sc)
        


In [22]:
# compute coherence value at various values of alpha and num_topics
def compute_coherence_values(documents, samp_size, prefix, num_topics_range, latent_dims_range, 
                             method = 'LDA_BERT', activation='relu', epochs=200, 
                             batch_size=128, show_visualize=True):
    
    coherence_values = []
    silhouette_values = []
    model_list = []
    for latent_dim in latent_dims_range:
        for num_topics in num_topics_range:
            prefix = '{}_{}_{}'.format(prefix, num_topics, latent_dim)
            lda_model, coherence_sc, silhouette_sc = main(documents, samp_size, prefix=prefix, ntopic=num_topics, method=method, 
                                                          latent_dim=latent_dim, activation=activation, epochs=epochs, batch_size=batch_size, show_visualize=show_visualize)
            model_list.append(lda_model)
            coherence_values.append((latent_dim, num_topics, coherence_sc))
            silhouette_values.append((latent_dim, num_topics, silhouette_sc))
            print()
        

    return model_list, coherence_values, silhouette_values

In [ ]:
prefix='india_based' 
activation='relu'
epochs=100
batch_size=128
method = "LDA_BERT"
samp_size=10000 # len(documents.index)
show_visualize=False

num_topics_range = [2, 6, 10, 15, 20]
latent_dims_range = [16, 32, 64, 128]

# num_topics_range = [2]
# latent_dims_range = [128]

model_list, coherence_values, silhouette_values = compute_coherence_values(india_based_documents, samp_size, prefix, num_topics_range, 
                         latent_dims_range, method, activation, epochs, batch_size,
                         show_visualize)

Preprocessing raw texts ...
Total samples : 10000
Preprocessing raw texts. Done!
Total preprocessed sentences : 9170
Total associated sentiments : 9170
Clustering embeddings ...
Getting vector representations for LDA ...
Getting vector representations for LDA. Done!
Getting vector representations for BERT ...


In [ ]:
coherence_df = pd.DataFrame(coherence_values, columns=['alpha', 'num_topics', 'coherence_value'])
coherence_df

In [ ]:
outside_india_based_tm = main(outside_india_based_documents)

Preprocessing raw texts ...
Preprocessing raw texts. Done!
Clustering embeddings ...
Getting vector representations for LDA ...
Getting vector representations for LDA. Done!
Getting vector representations for BERT ...



Getting vector representations for BERT. Done!
Fitting Autoencoder ...


In [24]:
tm = model_list[0]

In [28]:
%cd /content/drive/My Drive/tweets_analysis

!mkdir working

/content/drive/My Drive/tweets_analysis


In [31]:
%xmode Verbose
file_pi = open("/content/drive/My Drive/tweets_analysis/working/{}.file".format(tm.id), "w") 
pickle.dump(tm, file_pi)

Exception reporting mode: Verbose


TypeError: ignored

In [ ]:
file_pi = open("/content/drive/My Drive/tweets_analysis/working/{}.file".format(tm.id), "w") 
pickle.dump(outside_india_based_tm, file_pi)

# Step 6: Calculate abstraction and expression for each narrative 
Note that each cluster represents a narrative

In [ ]:
tweets_to_consider = 'fully_cleaned_tweet'

In [ ]:
final_clusters = np.unique(dfUnique['cl_num'])
print(final_clusters)

[ 1  2  3  5  7  8  9 10]


In [ ]:
#Store all tweets corresponding to each cluster in a file
for i in final_clusters:
    with open('./tweets_Cluster_'+str(i)+'.txt','w') as out:
        y = ''
        for x in dfUnique[tweets_to_consider][dfUnique.cl_num == i]:    
            y = y + x + '. '
        out.write(y)
        out.close()

In [ ]:
#A combination of (Noun, adjective, cardinal number, foreign word and Verb) are being extracted now
#Extract chunks matching pattern. Patterns are:
#1) Noun phrase (2 or more nouns occurring together. Ex United states of America, Abdul Kalam etc)
#2) Number followed by Noun (Ex: 28 Terrorists, 45th President)
#3) Adjective followed by Noun (Ex: Economic impact, beautiful inauguration)
#4) Foreign word (Ex: Jallikattu, Narendra modi, Pappu)
#5) Noun followed by Verb (Ex: Terrorists arrested)
#And a combination of all 5
        
import re
import nltk

phrases = pd.DataFrame({'extracted_phrases': [], 'cluster_num': []})


A = '(CD|JJ)/\w+\s'  #cd or jj
B = '(NN|NNS|NNP|NNPS)/\w+\s'  #nouns
C = '(VB|VBD|VBG|VBN|VBP|VBZ)/\w+\s' #verbs
D = 'FW/\w+\s'  #foreign word
patterns = ['('+A+B+')+', '('+D+B+')+','('+D+')+', '('+B+')+', '('+D+A+B+')+', 
           '('+B+C+')+', '('+D+B+C+')+', '('+B+A+B+')+', '('+B+B+C+')+'] 


def extract_phrases(tag1, tag2, sentences):
    extract_phrase = []
    for sentence in sentences:
        phrase = []
        next_word = 0
        for word, pos in nltk.pos_tag(nltk.word_tokenize(sentence)):
            if next_word == 1:
                next_word = 0
                if pos == tag2:
                    extract_phrase = np.append(extract_phrase,phrase + ' ' + word) 
            
            if pos == tag1:
                next_word = 1
                phrase = word
    return extract_phrase

for i in cluster_name:
    File = open('./tweets_Cluster_'+str(i)+'.txt', 'r') #open file
    lines = File.read() #read all lines
    sentences = nltk.sent_tokenize(lines) #tokenize sentences

    for sentence in sentences: 
        f = nltk.pos_tag(nltk.word_tokenize(sentence))
        tag_seq = []
        for word, pos in f:
            tag_seq.append(pos+'/'+ word)
        X = " ".join(tag_seq)

        phrase = []
        for j in range(len(patterns)):
            if re.search(patterns[j], X):
                phrase.append(' '.join([word.split('/')[1] for word in re.search(patterns[j], X).group(0).split()]))
    
        k = pd.DataFrame({'extracted_phrases': np.unique(phrase), 'cluster_num': int(i)})
    
        phrases = pd.concat([phrases,k], ignore_index = True)

print(phrases)

      cluster_num              extracted_phrases
0             1.0   centerofright demonetization
1             1.0                 demonetization
2             1.0                      scale has
3             1.0                 demonetization
4             1.0                 demonetization
5             1.0                 demonetization
6             1.0                 demonetization
7             1.0                 demonetization
8             1.0                   huge support
9             1.0             nation 8086 people
10            1.0                        support
11            1.0                 demonetization
12            1.0                 demonetization
13            1.0                 eliminate jobs
14            1.0                   huge support
15            1.0             nation 8086 people
16            1.0                        support
17            1.0                    money daily
18            1.0                   huge support
19            1.0   

### Keeping the largest phrase

In [ ]:
#For each phrase identified replace all the substrings by the largest phrase 
#Ex: lakh looted,40 lakh looted and Rs 40 lakh looted, replace all by single largest phrase - Rs 40 lakh looted 
#i.e. instead of 3 different phrases, there will be only one large phrase

phrases_final = pd.DataFrame({'extracted_phrases': [], 'cluster_num': []})
for i in cluster_name:
    phrases_for_each_cluster = []
    cluster_phrases = phrases['extracted_phrases'][phrases.cluster_num == i]
    cluster_phrases = np.unique(np.array(cluster_phrases))
    for j in range(len(cluster_phrases)):
        
        phrase = cluster_phrases[j]
        updated_cluster_phrases = np.delete((cluster_phrases), j)
        if any(phrase in phr for phr in updated_cluster_phrases): 
            'y'
        else: 
            #considering phrases of length greater than 1 only
            if (len(phrase.split(' '))) > 1:
                phrases_for_each_cluster.append(phrase)
    k = pd.DataFrame({'extracted_phrases': phrases_for_each_cluster, 'cluster_num': int(i) })
    
    phrases_final = pd.concat([phrases_final,k], ignore_index = True)

In [ ]:
phrases_final

,cluster_num,extracted_phrases
0,1.0,1st phase
1,1.0,93 support
2,1.0,abcyoutubech are
3,1.0,ananthkumar bjp
4,1.0,arshadwarsi thanks
5,1.0,bjp modi has
6,1.0,byelection bjp has
7,1.0,bypolls bjp has
8,1.0,centerofright demonetization
9,1.0,common people


In [ ]:
phrases_final.columns

Index(['cluster_num', 'extracted_phrases'], dtype='object')

### For each phrase in each cluster, calculate term frequency 

In [ ]:
#Term-frequency : For each cluster, calculate the number of times a given phrase occur in the tweets of that cluster

phrases_final['term_freq'] = len(phrases_final)*[0]

for i in cluster_name:
    for phrase in phrases_final['extracted_phrases'][phrases_final.cluster_num == i]:
        tweets = dfUnique[tweets_to_consider][dfUnique.cl_num == i]
        for tweet in tweets:
            if phrase in tweet:
                phrases_final['term_freq'][(phrases_final.extracted_phrases == phrase) & (phrases_final.cluster_num == i)] = phrases_final['term_freq'][(phrases_final.extracted_phrases == phrase) & (phrases_final.cluster_num == i)] + 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


### For each phrase in each cluster, calculate document frequency

In [ ]:
#Document-frequency
phrases_final['doc_freq'] = len(phrases_final)*[0]


# for each phrase, compute the number of clusters that Sphrase occurs in
for phrase in phrases_final['extracted_phrases']:
    for i in cluster_name:
        all_tweets = ''
        for tweet in dfUnique[tweets_to_consider][dfUnique.cl_num == i]:
            all_tweets = all_tweets + tweet + '. ' 
        if phrase in all_tweets:
            phrases_final['doc_freq'][(phrases_final.extracted_phrases == phrase) & (phrases_final.cluster_num == i)] = phrases_final['doc_freq'][(phrases_final.extracted_phrases == phrase) & (phrases_final.cluster_num == i)] + 1
        

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [ ]:
import math
phrases_final['doc_freq'] = phrases_final['doc_freq'].apply(lambda x: math.log10(n_best_clusters/(x)) )

### For each phrase in each cluster, calculate tf-idf

In [ ]:
phrases_final['tf-idf'] = phrases_final['term_freq']*phrases_final['doc_freq']

In [ ]:
phrases_final

,cluster_num,extracted_phrases,term_freq,doc_freq,tf-idf
0,1.0,1st phase,1,1.041393,1.041393
1,1.0,93 support,1,0.740363,0.740363
2,1.0,abcyoutubech are,1,1.041393,1.041393
3,1.0,ananthkumar bjp,1,1.041393,1.041393
4,1.0,arshadwarsi thanks,1,1.041393,1.041393
5,1.0,bjp modi has,1,1.041393,1.041393
6,1.0,byelection bjp has,2,1.041393,2.082785
7,1.0,bypolls bjp has,1,1.041393,1.041393
8,1.0,centerofright demonetization,1,0.564271,0.564271
9,1.0,common people,1,1.041393,1.041393


### For each cluster find top few phrases and respective sentiment
 

In [ ]:
phrases_final['diff_tf-idf'] = len(phrases_final)*[0]

narrative = pd.DataFrame({'cl_num': [], 'abstraction': []})
for i in cluster_name: 
    # arrange in descending order of tf-idf score
    phrases_final = phrases_final.sort_values(['cluster_num','tf-idf'], ascending=[1,0])
    
    #Break this distribution at a point where the difference between any consecutive phrases is maximum
    #difference between consecutive values of tf-idf 
    phrases_final['diff_tf-idf'][phrases_final.cluster_num == i] = abs(phrases_final['tf-idf'][phrases_final.cluster_num == i] - phrases_final['tf-idf'][phrases_final.cluster_num == i].shift(1))

    #The last value for each cluster will be 'NaN'. Replacing it with '0'. 
    phrases_final = phrases_final.fillna(0)
    
    phrases_final = phrases_final.reset_index(drop = True) #to avoid old index being added as a new column
    if len(phrases_final[phrases_final.cluster_num == i]) != 0:
        
        #index corresponding to the highest difference
 
        ind = (phrases_final['diff_tf-idf'][phrases_final.cluster_num == i]).idxmax()
        
        abstract = phrases_final['extracted_phrases'][:ind+1][phrases_final.cluster_num == i]
    
    
        #store the abstraction corresponding to each cluster
        k = pd.DataFrame({'cl_num': int(i), 'abstraction': abstract})
        narrative = pd.concat([narrative,k], ignore_index = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
dfUnique

,tweet,cleaned_tweet,fully_cleaned_tweet,sentiment,tokenized_tweet,cl_num,freq
2500,RT @centerofright: #DeMonetization - Finally -...,centerofright demonetization finally coun...,centerofright demonetization finally country s...,1,"[centerofright, demonetization, finally, count...",1,27
3136,The latest The Demonetization Daily! https://t...,the latest the demonetization daily,the latest the demonetization daily,1,"[the, latest, the, demonetization, daily]",1,1
3137,The latest The Demonetization Daily! https://t...,the latest the demonetization daily,the latest the demonetization daily,1,"[the, latest, the, demonetization, daily]",1,1
3138,The latest The Demonetization Daily! https://t...,the latest the demonetization daily,the latest the demonetization daily,1,"[the, latest, the, demonetization, daily]",1,1
3139,The latest The Demonetization Daily! https://t...,the latest the demonetization daily,the latest the demonetization daily,1,"[the, latest, the, demonetization, daily]",1,1
3140,The latest The Demonetization Daily! https://t...,the latest the demonetization daily,the latest the demonetization daily,1,"[the, latest, the, demonetization, daily]",1,1
1466,Huge support for #PM #NarendraModii ’s #demone...,huge support for narendramodii demonetiz...,huge support for narendramodii demonetization ...,1,"[huge, support, for, narendramodii, demonetiza...",1,1
3141,The latest The Demonetization Daily! https://t...,the latest the demonetization daily,the latest the demonetization daily,1,"[the, latest, the, demonetization, daily]",1,1
3142,The latest The Demonetization Daily! https://t...,the latest the demonetization daily,the latest the demonetization daily,1,"[the, latest, the, demonetization, daily]",1,1
3143,The latest The I Eliminate Jobs Daily! https:/...,the latest the eliminate jobs daily,the latest the eliminate jobs daily,1,"[the, latest, the, eliminate, jobs, daily]",1,1


In [ ]:
#Assigning polarity based on the sentiment for each tweet 2=negative, 1=positive, 3=neutral
dfUnique['polarity'] = np.NaN
dfUnique['polarity'][dfUnique.sentiment == 0.5] = "3"
dfUnique['polarity'][dfUnique.sentiment == 1] = "1"
dfUnique['polarity'][dfUnique.sentiment == 0] = "2"

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


### Assign the sentiment to each extracted phrases
count the number of tweets, a phrase has occurred in positive, negative and neutral context. Assign the most occurred sentiment to the phrase

In [ ]:
from collections import Counter

#find the highest occurring sentiment corresponding to each tweet
def find_mode(a):
    b = Counter(a).most_common(3)
    mode = []; c_max = 0
    for a,c in b:
        if c>c_max:
            c_max = c
        if c_max == c:
            mode.append(a)  
    print(mode)
    mode.sort()
    print(mode)
    
    ## if mode is 3&2 i.e. neutral and negative, assign the overall sentiment for that phrase as negative, 
    ## if mode is 3&1 i.e. neutral and positive, assign the overall sentiment for that phrase as positive,
    ## if mode is 2&1 i.e. negative and positive, assign the overall sentiment for that phrase as neutal, 
    ## if mode is 3&2&1 i.e. negative, positive and neutral, assign the overall sentiment for that phrase as neutral
    
    if len(mode) == 1:
        return mode[0]
    
    elif (len(mode) == 2) & (mode[1]=='3'):
        return mode[0]
    else:
        return 3
    
#1=>+ve 2=>-ve 3=>Neutral
narrative['expression'] = -1
dfUnique = dfUnique.reset_index(drop = True)
for i in cluster_name:
    tweets = dfUnique[tweets_to_consider][dfUnique.cl_num == i]
    abstracts = narrative['abstraction'][narrative.cl_num == i] 
    for abst in abstracts:
        sent = []
        for tweet, polarity in zip(dfUnique[tweets_to_consider][dfUnique.cl_num == i], dfUnique['polarity'][dfUnique.cl_num == i]):
            if abst in tweet:
                sent = np.append(sent, polarity)
        
        
        if len(sent)!=0:
            ## if mode is 3&2-2, 3&1-1, 2&1-3, 3&2&1 - 3
            senti = find_mode(sent)
            if senti == '2':
                sent_value = "Negative"
            elif senti == '1':
                sent_value = "Positive"
            else:
                sent_value = "Neutral"
            narrative['expression'][(narrative.abstraction == abst) & (narrative.cl_num == i)] = sent_value
        

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['1', '2']
['1', '2']
['1']
['1']
['1']
['1']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['1']
['1']
['1']
['1']
['1']
['1']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']




# Save the narratives in excel file
 With each sheet in the file representing 1 narrative ( == 1 cluster)

In [ ]:
#sudo pip install xlwt
#sudo pip3 install openpyxl
from pandas import ExcelWriter

#Save the narratives in an excel file 

writer = pd.ExcelWriter('narrative.xlsx')
for i in cluster_name:
    df1 = pd.DataFrame(dfUnique[['tweet','freq']][dfUnique.cl_num == i]).sort_values(['freq'], ascending = [0])
    df1 = pd.DataFrame({'tweet': dfUnique['tweet'][dfUnique.cl_num == i], 'freq': dfUnique['freq'][dfUnique.cl_num == i]}) 
    df1 = df1.sort_values(['freq'], ascending = [0]) 

    df2 = pd.DataFrame({ 'abstraction': narrative['abstraction'][narrative.cl_num == i], 'expression': narrative['expression'][narrative.cl_num == i]})
    df3 = pd.DataFrame({'abstraction': (len(df1)-len(df2))*['-'], 'expression': (len(df1)-len(df2))*['-']})
    df2 = df2.append(df3)

    df1 = df1.reset_index(drop=True)
    df2 = df2.reset_index(drop=True)
    df1['abstraction'] = df2['abstraction']
    df1['expression'] = df2['expression']

    df1.to_excel(writer,'narrative_cluster'+str(i))

writer.save()
    

In [ ]:
narrative

,abstraction,cl_num,expression
0,support demonetization,1.0,Neutral
1,nation 8086 people,1.0,Positive
2,modibharosa huge support,1.0,Positive
3,demonetization issue,2.0,Negative
4,thedarjchron demonetization northeast reaction,2.0,Negative
5,system india article,3.0,Negative
6,overall impact,3.0,Negative
7,shop owner gets,3.0,Negative
8,politics nitishkumar supports,5.0,Negative
9,coop banks,5.0,Negative


### TODO:

* Implement models.ldamulticore – parallelized Latent Dirichlet Allocation using all CPU cores to parallelize and speed up model training.
* Switch from BERT/RoBERTa to SciBERT, BART, and or other models. 